In [ ]:
include("../src/MLTrain.jl")
include("../src/Data_prep.jl")
include("../src/Utils.jl")


function build_toy_branch(Atom_name::String,G1_number::Int,R_cutoff::Float32)
    ion_charge = 0.1f0 * element_to_charge[Atom_name]
    return Chain(
        MyLayer(1, G1_number, R_cutoff, ion_charge),
        Dense(G1_number, 1)
    )
end


function create_toy_species_models(
    unique_species::Vector{String},
    G1_number::Int,
    R_cutoff::Float32
)
    models = Dict{String, Chain}()
    for sp in unique_species
        # build_branch returns a Flux.Chain for that species
        models[sp] = build_toy_branch(sp, G1_number, R_cutoff)
    end
    return models
end




create_toy_species_models (generic function with 1 method)

In [ ]:
file_path = "reduced_train.xyz"
    
# Step 1: Extract information from the input file
N_atoms, species, unique_species, all_cells, dataset, all_energies = extract_data(file_path)

    
# Step 2: Create neural network input
nn_input_dataset = create_nn_input(dataset, N_atoms)



# Step 3: Create neural network target
 target = create_nn_target(dataset, all_energies)


# Step 4: Data preprocessing
Train, Val, _, _, _, _ = data_preprocess(nn_input_dataset, target,split=[0.6, 0.2, 0.2])





((Float32[3.0717585 -0.14971268 3.0603335; 3.210517 -0.16328853 3.0112257; 3.5257893 -0.04356769 3.2849522;;; 3.03046 -0.04373725 0.010446698; 3.1080205 0.08581428 -0.24876891; 3.029456 0.03864127 0.10844886;;; 2.877253 -0.025660131 -0.40077204; 3.0260594 -0.054103684 -0.2785623; 3.4833195 -0.21320987 -0.19319236], Dict{Symbol, Any}[Dict(:energy => -0.86057436f0, :forces => Float32[1.5904154 1.5511401 -0.28411958; 0.3162832 0.9061826 1.5709159; -0.57991886 0.25863886 -0.32083106]), Dict(:energy => 1.4021714f0, :forces => Float32[0.30106702 0.35098273 0.20707221; -1.131774 -1.9106978 -1.7895982; -0.9700257 0.87433493 0.71551895]), Dict(:energy => -0.541597f0, :forces => Float32[0.39772835 0.40516022 -0.103471845; 0.21879585 0.10554808 0.38255933; -2.5977683 -0.4391377 -0.024999648])]), (Float32[3.210517 -0.16328853 3.0112257;;; 3.1080205 0.08581428 -0.24876891;;; 3.0260594 -0.054103684 -0.2785623], Dict{Symbol, Any}[Dict(:energy => 1.4054941f0, :forces => Float32[0.30106702 0.35098273 0

In [ ]:
# Step 5: Create branches
species_models = create_toy_species_models(species,1,5.0f0)   


# Step 6: Create models
model = assemble_model(species_models, species,all_cells[1])


x_sample= Train[1][1,:,:]

y_sample=Train[2][1]


println("x_sample: ",x_sample)
println("x_sample dims: ",size(x_sample))
println("y_sample: ",y_sample)

println("model output: ",model(x_sample))


println("model loss on the sample: ",loss_function(model,x_sample,y_sample))

x_sample: Float32[3.0717585 3.03046 2.877253; -0.14971268 -0.04373725 -0.025660131; 3.0603335 0.010446698 -0.40077204]
x_sample dims: (3, 3)
y_sample: Dict{Symbol, Any}(:energy => -0.86057436f0, :forces => Float32[1.5904154 1.5511401 -0.28411958; 0.3162832 0.9061826 1.5709159; -0.57991886 0.25863886 -0.32083106])

In [ ]:
    # Step 7: Train the model

    time_train = @elapsed trained_model,train_loss,val_loss = train_model!(
        model,
        Train[1],  
        Train[2], 
        Val[1],
        Val[2],
        loss_function;
         initial_lr=0.1,epochs=1, batch_size=4, verbose=false
    )

In [ ]:
distances=distance_layer(x_sample[1,:,:],1,nothing)

println(distances)

jacob = Zygote.jacobian(x->distance_layer(x,1,nothing),x_sample[1,:,:])

println(jacob)

Float32[5.669461 4.606348]
(Float32[0.5640261 -0.5640261 0.0 0.54643726 -0.54643726 0.0 0.6190969 -0.6190969 0.0; 0.015365967 0.0 -0.015365967 0.80255055 0.0 -0.80255055 0.5963862 0.0 -0.5963862],)


In [ ]:
First_layer=Parallel(
    vcat,
    x -> distance_layer(x, 1, nothing),
    x -> distance_layer(x, 2, nothing),
    x -> distance_layer(x, 3, nothing)
)



First_layer(x_sample[1,:,:])

jacob=Zygote.jacobian(x->First_layer(x),x_sample[1,:,:])

println(jacob)


(Float32[0.5640261 -0.5640261 0.0 0.54643726 -0.54643726 0.0 0.6190969 -0.6190969 0.0; 0.5640261 -0.5640261 0.0 0.54643726 -0.54643726 0.0 0.6190969 -0.6190969 0.0; 0.015365967 0.0 -0.015365967 0.80255055 0.0 -0.80255055 0.5963862 0.0 -0.5963862; 0.015365967 0.0 -0.015365967 0.80255055 0.0 -0.80255055 0.5963862 0.0 -0.5963862; 0.0 -0.9551224 0.9551224 0.0 0.1829099 -0.1829099 0.0 -0.23299156 0.23299156; 0.0 -0.9551224 0.9551224 0.0 0.1829099 -0.1829099 0.0 -0.23299156 0.23299156],)


In [ ]:
println(First_layer)
println(model)

Parallel(#169, #170, #171)
Chain(Parallel(vcat, Chain(#2, Chain(MyLayer(Float32[0.86726224;;], Float32[0.9185761;;], 5.0f0, 5.5f0), Dense(1 => 1))), Chain(#2, Chain(MyLayer(Float32[1.9316505;;], Float32[2.0077033;;], 5.0f0, 8.2f0), Dense(1 => 1))), Chain(#2, Chain(MyLayer(Float32[0.6461651;;], Float32[1.701741;;], 5.0f0, 5.3f0), Dense(1 => 1)))), #3)


In [ ]:
println(distances)

jacob = Zygote.jacobian(x->distance_layer(x,2,nothing),x_sample[1,:,:])

println(jacob)

In [ ]:
layer=MyLayer(1,1,5.95f0,82.0f0)

all_forces = [Zygote.gradient(x->layer(x)[1],distances[i,:]) for i in 1:2]

1-element Vector{Float32}:
 0.00035513175